<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>

# **Fine-Tuning Transformers with PyTorch and Hugging Face**

Estimated time needed: **45** minutes

# Introduction

This project aims to introduce you to the process of loading and fine-tuning pretrained large language models (LLMs) 

You will learn how to implement the training loop of a model using pytorch to tune a model on task-specific data, as well as fine-tuning a model on task-specific data using the SFTTrainer module from Hugging Face. Finally, you will learn how to evaluate the performance of the fine-tuned models.

By the end of this project, you will have a solid understanding of how to leverage pretrained LLMs and fine-tune them for your specific use cases, empowering you to create powerful and customized natural language processing solutions.

# __Table of Contents__

<ol>
    <li><a href="#Objectives">Objectives</a></li>
    <li>
        <a href="#Setup">Setup</a>
        <ol>
            <li><a href="#Installing-required-libraries">Installing required libraries</a></li>
            <li><a href="#Importing-required-libraries">Importing required libraries</a></li>
        </ol>
    </li>
    <li><a href="#Supervised-Fine-tuning-with-PyTorch)">Supervised Fine-tuning with Pytorch</a>
        <ol>
            <li><a href="#Dataset-preparations">Dataset preparations</a></li>
            <li><a href="#Train-the-model">Train the model</a></li>
            <li><a href="#Evaluate">Evaluate</a></li>
            <li><a href="#Loading-the-saved-model">Loading the saved model</a></li>
        </ol>
    </li>
    <li><a href="#Exercise:-Training-a-conversational-model-using-SFTTrainer">Exercise: Training a conversational model using SFTTrainer</a></li>
</ol>

---

# Objectives

After completing this lab, you will be able to:

 - Load pretrained LLMs from Hugging Face and make inferences
 - Fine-tune a model on task-specific data using the SFTTrainer module from Hugging Face
 - Load a SFTTrainer pretrained model and make comparisons
 - Evaluate the model

---

# Setup

### Installing required libraries

The following required libraries are pre-installed in the Skills Network Labs environment. However, if you run this notebook commands in a different Jupyter environment (e.g. Watson Studio or Ananconda), you will need to install these libraries by removing the `#` sign before `!pip` in the code cell below.


In [1]:
# All Libraries required for this lab are listed below. The libraries pre-installed on Skills Network Labs are commented.
# !pip install -qy pandas==1.3.4 numpy==1.21.4 seaborn==0.9.0 matplotlib==3.5.0 torch=2.1.0+cu118
# - Update a specific package
# !pip install pmdarima -U
# - Update a package to specific version
# !pip install --upgrade pmdarima==2.0.2
# Note: If your environment doesn't support "!pip install", use "!mamba install"

The following required libraries are __not__ pre-installed in the Skills Network Labs environment. __You will need to run the following cell__ to install them:

```bash
!pip install transformers==4.42.1
!pip install datasets # 2.20.0
!pip install portalocker>=2.0.0
!pip install torch==2.3.1
!pip install torchmetrics==1.4.0.post0
#!pip install numpy==1.26.4
#!pip install peft==0.11.1
#!pip install evaluate==0.4.2
#!pip install -q bitsandbytes==0.43.1
!pip install accelerate==0.31.0
!pip install torchvision==0.18.1


!pip install trl==0.9.4
!pip install protobuf==3.20.*
!pip install matplotlib

```

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"]="0"
# Set the environment variable TOKENIZERS_PARALLELISM to 'false'
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())


from torchmetrics import Accuracy
from torch.optim.lr_scheduler import LambdaLR
from torch.utils.data import DataLoader
from torch.optim import AdamW
from transformers import AutoConfig,AutoModelForCausalLM,AutoModelForSequenceClassification,BertConfig,BertForMaskedLM,TrainingArguments, Trainer, TrainingArguments
from transformers import AutoTokenizer,BertTokenizerFast,TextDataset,DataCollatorForLanguageModeling
from transformers import pipeline
from datasets import load_dataset
from trl import SFTConfig,SFTTrainer, DataCollatorForCompletionOnlyLM


#import numpy as np
#import pandas as pd
from tqdm.auto import tqdm
import math
import time
import matplotlib.pyplot as plt
#import pandas as pd


# You can also use this section to suppress warnings generated by your code:
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
warnings.filterwarnings('ignore')

print("Import Successfully!")

True
Tesla P40
Import Successfully!


---

# Supervised Fine-tuning with Pytorch

Fine-tuning Transformers, specifically BERT (Bidirectional Encoder Representations from Transformers), refers to the process of training a pretrained BERT model on a specific downstream task. BERT is an encoder-only language model that has been pretrained on a large corpus of text to learn contextual representations of words.

Fine-tuning BERT involves taking the pretrained model and further training it on a task-specific dataset, such as sentiment analysis or question answering. During fine-tuning, the parameters of the pretrained BERT model are updated and adapted to the specifics of the target task.

This process is important because it allows you to leverage the knowledge and language understanding captured by BERT and apply it to different tasks. By fine-tuning BERT, you can benefit from its contextual understanding of language and transfer that knowledge to specific domain-specific or task-specific problems. Fine-tuning enables BERT to learn from a smaller labeled dataset and generalize well to unseen examples, making it a powerful tool for various natural language processing tasks. It helps to bridge the gap between pretraining on a large corpus and the specific requirements of downstream applications, ultimately improving the performance and effectiveness of models in various real-world scenarios.


## Dataset preparations

The Yelp review dataset is a widely used dataset in natural language processing (NLP) and sentiment analysis research. It consists of user reviews and accompanying metadata from the Yelp platform, which is a popular online platform for reviewing and rating local businesses such as restaurants, hotels, and shops.

The dataset includes 6,990,280 reviews written by Yelp users, covering a wide range of businesses and locations. Each review typically contains the text of the review itself alongwith the star rating given by the user (ranging from 1 to 5).

Our aim in this lab, is to fine-tune a pretrained BERT model to predict the ratings from reviews.


Let's start by loading the yelp_review data:


In [3]:
dataset = load_dataset("yelp_review_full")

dataset

README.md:   0%|          | 0.00/6.72k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/299M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/23.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['label', 'text'],
        num_rows: 650000
    })
    test: Dataset({
        features: ['label', 'text'],
        num_rows: 50000
    })
})